In [1]:
#라이브러리 임포트 
from sklearn import datasets
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd

from xgboost import XGBRegressor, XGBClassifier
from catboost import CatBoostRegressor, CatBoostClassifier
from lightgbm import LGBMRegressor,LGBMClassifier

np.random.seed(5)

class CFG:
    SEED = 42

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
train_df = pd.DataFrame(train)
test_df = pd.DataFrame(test)

In [4]:
x_train =  train_df[['trait', 'SNP_01', 'SNP_02', 'SNP_03', 'SNP_04', 'SNP_05', 'SNP_06', 'SNP_07', 'SNP_08', 'SNP_09',
                    'SNP_10', 'SNP_11', 'SNP_12', 'SNP_13', 'SNP_14', 'SNP_15',]]
y_train = train_df[['class']]

x_test =  test_df[['trait', 'SNP_01', 'SNP_02', 'SNP_03', 'SNP_04', 'SNP_05', 'SNP_06', 'SNP_07', 'SNP_08', 'SNP_09',
                    'SNP_10', 'SNP_11', 'SNP_12', 'SNP_13', 'SNP_14', 'SNP_15',]]

## Data Pre-processing
### Label-Encoding

In [5]:
class_le = preprocessing.LabelEncoder()
snp_le = preprocessing.LabelEncoder()
snp_col = [f'SNP_{str(x).zfill(2)}' for x in range(1,16)]

In [6]:
snp_data = []
for col in snp_col:
    snp_data += list(x_train[col].values)

In [7]:
y_train = class_le.fit_transform(y_train)
snp_le.fit(snp_data)

C:\ProgramData\Anaconda3\envs\tf_cpu\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LabelEncoder()

In [8]:
for col in x_train.columns:
    if col in snp_col:
        x_train[col] = snp_le.transform(x_train[col])
        x_test[col] = snp_le.transform(x_test[col])

C:\ProgramData\Anaconda3\envs\tf_cpu\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\envs\tf_cpu\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [9]:
x_train.dtypes

trait     int64
SNP_01    int32
SNP_02    int32
SNP_03    int32
SNP_04    int32
SNP_05    int32
SNP_06    int32
SNP_07    int32
SNP_08    int32
SNP_09    int32
SNP_10    int32
SNP_11    int32
SNP_12    int32
SNP_13    int32
SNP_14    int32
SNP_15    int32
dtype: object

## 단일 모델 정확도 측정
### 의사결정트리, knn, svm 모델의 정확도 측정.

In [10]:
dtree = DecisionTreeClassifier(random_state= CFG.SEED) #그리드서치해서 나온값

dtree = dtree.fit(x_train, y_train)

dtree_predicted = dtree.predict(x_test)

dtree_predicted

array([0, 1, 2, 1, 0, 1, 2, 2, 0, 0, 2, 1, 1, 0, 1, 1, 0, 1, 1, 2, 1, 1,
       2, 0, 1, 2, 1, 0, 0, 1, 0, 0, 1, 2, 0, 2, 2, 1, 1, 2, 0, 1, 2, 1,
       1, 1, 1, 2, 1, 2, 0, 1, 0, 1, 1, 1, 2, 0, 1, 2, 0, 1, 2, 2, 2, 0,
       1, 0, 0, 1, 1, 1, 0, 0, 2, 1, 2, 1, 1, 1, 2, 1, 0, 1, 1, 1, 1, 1,
       2, 0, 1, 1, 2, 1, 1, 2, 0, 1, 0, 2, 0, 1, 1, 2, 0, 0, 2, 1, 0, 1,
       2, 1, 1, 1, 1, 0, 0, 2, 1, 1, 0, 1, 1, 2, 2, 1, 2, 1, 0, 1, 0, 0,
       1, 1, 1, 2, 0, 0, 1, 0, 0, 0, 2, 1, 1, 2, 0, 1, 2, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 2, 0, 1, 2, 2, 1, 0, 0, 2, 1, 1, 0, 1, 2, 2, 1, 1])

In [11]:
knn = KNeighborsClassifier(n_neighbors = 26).fit(x_train, y_train) 
knn_predicted = knn.predict(x_test) 
knn_predicted

array([0, 1, 2, 1, 0, 1, 2, 1, 0, 0, 2, 1, 1, 0, 1, 1, 0, 1, 1, 2, 1, 1,
       1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 2, 1, 1, 2, 0, 1, 2, 1,
       1, 1, 1, 1, 1, 2, 0, 1, 0, 1, 1, 1, 2, 0, 1, 2, 0, 1, 2, 2, 2, 0,
       1, 0, 0, 1, 1, 1, 0, 0, 2, 1, 2, 1, 1, 1, 2, 1, 0, 1, 1, 1, 1, 1,
       2, 0, 1, 1, 1, 1, 1, 2, 0, 1, 0, 2, 0, 1, 1, 2, 0, 0, 2, 1, 0, 1,
       2, 1, 1, 1, 1, 0, 0, 2, 1, 1, 0, 1, 1, 2, 2, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 2, 0, 0, 1, 0, 0, 0, 2, 1, 1, 1, 0, 1, 2, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 2, 0, 1, 2, 2, 1, 0, 0, 2, 1, 1, 0, 1, 2, 2, 1, 1])

In [12]:
svm = SVC(C=0.1, gamma=0.003, probability = True, random_state = CFG.SEED).fit(x_train, y_train) #그리드서치해서 나온값
svm_predicted = svm.predict(x_test)
svm_predicted

array([0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 2, 0, 1, 2, 0, 1, 1, 1, 1, 0,
       1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 2, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 2, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1])

In [13]:
submit = pd.read_csv('./sample_submission.csv')

In [14]:
submit['class'] = class_le.inverse_transform(dtree_predicted)
submit.to_csv('./answer/submit_dtree.csv', index=False)

In [15]:
submit['class'] = class_le.inverse_transform(knn_predicted)
submit.to_csv('./answer/submit_knn.csv', index=False)

In [16]:
submit['class'] = class_le.inverse_transform(svm_predicted)
submit.to_csv('./answer/submit_svm.csv', index=False)

## 하드보팅

In [17]:
voting_clf = VotingClassifier(estimators=[('decision_tree', dtree), ('knn', knn), ('svm', svm)],
                                 weights=[1, 1, 1], voting ='hard').fit(x_train, y_train)
hard_voting_predicted = voting_clf.predict(x_test)
hard_voting_predicted

array([0, 1, 2, 1, 0, 1, 2, 1, 0, 0, 2, 1, 1, 0, 1, 1, 0, 1, 1, 2, 1, 1,
       1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 2, 1, 1, 2, 0, 1, 2, 1,
       1, 1, 1, 1, 1, 2, 0, 1, 0, 1, 1, 1, 2, 0, 1, 2, 0, 1, 2, 2, 2, 0,
       1, 0, 0, 1, 1, 1, 0, 0, 2, 1, 2, 1, 1, 1, 2, 1, 0, 1, 1, 1, 1, 1,
       2, 0, 1, 1, 1, 1, 1, 2, 0, 1, 0, 2, 0, 1, 1, 2, 0, 0, 2, 1, 0, 1,
       2, 1, 1, 1, 1, 0, 0, 2, 1, 1, 0, 1, 1, 2, 2, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 2, 0, 0, 1, 0, 0, 0, 2, 1, 1, 1, 0, 1, 2, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 2, 0, 1, 2, 2, 1, 0, 0, 2, 1, 1, 0, 1, 2, 2, 1, 1])

In [18]:
submit['class'] = class_le.inverse_transform(hard_voting_predicted)
submit.to_csv('./answer/submit_hardvoting.csv', index=False)

## 소프트보팅

In [19]:
voting_clf = VotingClassifier(estimators=[('decision_tree', dtree), ('knn', knn), ('svm', svm)],
                                 weights=[1, 1, 1], voting ='soft').fit(x_train, y_train)
soft_voting_predicted = voting_clf.predict(x_test)
soft_voting_predicted

array([0, 1, 2, 1, 0, 1, 2, 1, 0, 0, 2, 1, 1, 0, 1, 1, 0, 1, 1, 2, 1, 1,
       1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 2, 0, 2, 2, 1, 1, 2, 0, 1, 2, 1,
       1, 1, 1, 2, 1, 2, 0, 1, 0, 1, 1, 1, 2, 0, 1, 2, 0, 1, 2, 2, 2, 0,
       1, 0, 0, 1, 1, 1, 0, 0, 2, 1, 2, 1, 1, 1, 2, 1, 0, 1, 1, 1, 1, 1,
       2, 0, 1, 1, 2, 1, 1, 2, 0, 1, 0, 2, 0, 1, 1, 2, 0, 0, 2, 1, 0, 1,
       2, 1, 1, 1, 1, 0, 0, 2, 1, 1, 0, 1, 1, 2, 2, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 2, 0, 0, 1, 0, 0, 0, 2, 1, 1, 2, 0, 1, 2, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 2, 0, 1, 2, 2, 1, 0, 0, 2, 1, 1, 0, 1, 2, 2, 1, 1])

In [20]:
submit['class'] = class_le.inverse_transform(soft_voting_predicted)
submit.to_csv('./answer/submit_softvoting.csv', index=False)

## 랜덤포레스트 추가

In [21]:
ranfor = RandomForestClassifier(random_state=CFG.SEED)
ranfor = ranfor.fit(x_train, y_train)
ranfor_predicted = ranfor.predict(x_test)
ranfor_predicted

array([0, 1, 2, 1, 0, 1, 2, 1, 0, 0, 2, 1, 2, 0, 1, 1, 0, 1, 1, 2, 1, 1,
       1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 2, 0, 1, 2, 1, 1, 2, 0, 1, 2, 1,
       1, 1, 1, 2, 1, 2, 0, 1, 0, 1, 1, 1, 2, 0, 1, 2, 0, 1, 2, 2, 2, 0,
       1, 0, 0, 1, 1, 1, 0, 0, 2, 1, 2, 1, 1, 1, 2, 1, 0, 1, 1, 1, 1, 1,
       2, 0, 1, 1, 2, 1, 1, 2, 0, 1, 0, 2, 0, 1, 1, 2, 0, 0, 2, 1, 0, 1,
       2, 1, 1, 1, 1, 0, 0, 2, 1, 1, 0, 1, 1, 2, 2, 1, 2, 1, 0, 1, 0, 0,
       1, 1, 1, 2, 0, 0, 1, 0, 0, 0, 2, 1, 1, 1, 0, 1, 2, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 2, 0, 1, 2, 2, 1, 0, 0, 2, 1, 1, 0, 1, 2, 2, 1, 1])

## CatBoost 추가

In [22]:
catboost = CatBoostClassifier(random_state=CFG.SEED)
catboost = catboost.fit(x_train, y_train)
catboost_predicted = catboost.predict(x_test)
catboost_predicted

Learning rate set to 0.073902
0:	learn: 1.0289654	total: 132ms	remaining: 2m 11s
1:	learn: 0.9536345	total: 132ms	remaining: 1m 6s
2:	learn: 0.8977301	total: 133ms	remaining: 44.2s
3:	learn: 0.8531481	total: 134ms	remaining: 33.3s
4:	learn: 0.8065217	total: 134ms	remaining: 26.7s
5:	learn: 0.7650036	total: 135ms	remaining: 22.3s
6:	learn: 0.7272086	total: 135ms	remaining: 19.2s
7:	learn: 0.6899499	total: 136ms	remaining: 16.8s
8:	learn: 0.6575878	total: 137ms	remaining: 15s
9:	learn: 0.6244861	total: 137ms	remaining: 13.6s
10:	learn: 0.5922186	total: 138ms	remaining: 12.4s
11:	learn: 0.5658850	total: 138ms	remaining: 11.4s
12:	learn: 0.5389163	total: 139ms	remaining: 10.5s
13:	learn: 0.5179312	total: 139ms	remaining: 9.82s
14:	learn: 0.4959282	total: 140ms	remaining: 9.19s
15:	learn: 0.4770140	total: 141ms	remaining: 8.64s
16:	learn: 0.4575394	total: 141ms	remaining: 8.15s
17:	learn: 0.4422733	total: 142ms	remaining: 7.72s
18:	learn: 0.4256502	total: 142ms	remaining: 7.34s
19:	learn: 0

476:	learn: 0.0168381	total: 377ms	remaining: 413ms
477:	learn: 0.0167997	total: 378ms	remaining: 413ms
478:	learn: 0.0167634	total: 379ms	remaining: 412ms
479:	learn: 0.0167257	total: 379ms	remaining: 411ms
480:	learn: 0.0166901	total: 380ms	remaining: 410ms
481:	learn: 0.0166474	total: 380ms	remaining: 409ms
482:	learn: 0.0165945	total: 381ms	remaining: 408ms
483:	learn: 0.0165503	total: 381ms	remaining: 407ms
484:	learn: 0.0165003	total: 382ms	remaining: 406ms
485:	learn: 0.0164616	total: 382ms	remaining: 405ms
486:	learn: 0.0164152	total: 383ms	remaining: 403ms
487:	learn: 0.0163701	total: 384ms	remaining: 402ms
488:	learn: 0.0163117	total: 384ms	remaining: 401ms
489:	learn: 0.0162777	total: 385ms	remaining: 400ms
490:	learn: 0.0162462	total: 385ms	remaining: 399ms
491:	learn: 0.0162134	total: 386ms	remaining: 398ms
492:	learn: 0.0161744	total: 386ms	remaining: 397ms
493:	learn: 0.0161334	total: 387ms	remaining: 396ms
494:	learn: 0.0160962	total: 387ms	remaining: 395ms
495:	learn: 

834:	learn: 0.0088317	total: 557ms	remaining: 110ms
835:	learn: 0.0088189	total: 558ms	remaining: 109ms
836:	learn: 0.0088063	total: 558ms	remaining: 109ms
837:	learn: 0.0087985	total: 559ms	remaining: 108ms
838:	learn: 0.0087884	total: 560ms	remaining: 107ms
839:	learn: 0.0087788	total: 560ms	remaining: 107ms
840:	learn: 0.0087676	total: 561ms	remaining: 106ms
841:	learn: 0.0087580	total: 561ms	remaining: 105ms
842:	learn: 0.0087471	total: 562ms	remaining: 105ms
843:	learn: 0.0087332	total: 562ms	remaining: 104ms
844:	learn: 0.0087198	total: 563ms	remaining: 103ms
845:	learn: 0.0087091	total: 563ms	remaining: 103ms
846:	learn: 0.0086982	total: 564ms	remaining: 102ms
847:	learn: 0.0086854	total: 564ms	remaining: 101ms
848:	learn: 0.0086740	total: 565ms	remaining: 100ms
849:	learn: 0.0086630	total: 565ms	remaining: 99.8ms
850:	learn: 0.0086487	total: 566ms	remaining: 99.1ms
851:	learn: 0.0086372	total: 567ms	remaining: 98.4ms
852:	learn: 0.0086285	total: 567ms	remaining: 97.7ms
853:	lea

array([[0],
       [1],
       [2],
       [1],
       [0],
       [1],
       [2],
       [1],
       [0],
       [0],
       [2],
       [1],
       [2],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [2],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [2],
       [0],
       [1],
       [2],
       [1],
       [1],
       [2],
       [0],
       [1],
       [2],
       [1],
       [1],
       [1],
       [1],
       [2],
       [1],
       [2],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [2],
       [0],
       [1],
       [2],
       [0],
       [1],
       [2],
       [2],
       [2],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [2],
       [1],
       [2],
       [1],
       [1],
       [1],
       [2],
       [1],
       [0],
    

## XGB 추가

In [23]:
xgb = XGBClassifier(random_state=CFG.SEED)
xgb = xgb.fit(x_train, y_train)
xgb_predicted = xgb.predict(x_test)
xgb_predicted

array([0, 1, 2, 1, 0, 1, 2, 1, 0, 0, 2, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 2, 0, 1, 2, 1, 1, 2, 0, 1, 2, 1,
       1, 1, 1, 2, 1, 2, 0, 1, 0, 1, 1, 1, 2, 0, 1, 2, 0, 1, 2, 2, 2, 0,
       1, 0, 0, 1, 1, 1, 0, 0, 2, 1, 2, 1, 1, 1, 2, 1, 0, 1, 1, 1, 1, 1,
       2, 0, 1, 1, 2, 1, 1, 2, 0, 1, 0, 2, 0, 1, 1, 2, 0, 0, 2, 1, 0, 1,
       2, 1, 1, 1, 1, 0, 0, 2, 1, 1, 0, 1, 1, 2, 2, 1, 2, 1, 0, 1, 0, 0,
       1, 1, 1, 2, 0, 0, 1, 0, 0, 0, 2, 1, 1, 1, 0, 1, 2, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 2, 0, 1, 2, 2, 1, 0, 0, 2, 1, 1, 0, 1, 2, 2, 1, 1],
      dtype=int64)

## LGBM 추가

In [24]:
lgbm = LGBMClassifier(random_state=CFG.SEED)
lgbm = lgbm.fit(x_train, y_train)
lgbm_predicted = lgbm.predict(x_test)
lgbm_predicted

array([0, 1, 2, 1, 0, 1, 2, 1, 0, 0, 2, 1, 1, 0, 1, 1, 0, 1, 1, 2, 1, 1,
       1, 0, 1, 2, 1, 0, 0, 1, 0, 0, 1, 2, 0, 1, 2, 1, 1, 2, 0, 1, 2, 1,
       1, 1, 1, 2, 1, 2, 0, 1, 0, 1, 1, 1, 2, 0, 1, 2, 0, 1, 2, 2, 2, 0,
       1, 0, 0, 1, 1, 1, 0, 0, 2, 1, 2, 1, 1, 1, 2, 1, 0, 1, 1, 1, 1, 1,
       2, 0, 1, 1, 2, 1, 1, 2, 0, 1, 0, 2, 0, 1, 1, 2, 0, 0, 2, 1, 0, 1,
       2, 2, 1, 1, 1, 0, 0, 2, 1, 2, 0, 1, 1, 2, 2, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 2, 0, 0, 1, 0, 0, 0, 2, 1, 1, 1, 0, 1, 2, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 2, 0, 1, 2, 2, 1, 0, 0, 2, 1, 1, 0, 1, 2, 2, 1, 1])

In [27]:
voting_clf = VotingClassifier(estimators=[('random_foresting',ranfor),('catboost', catboost), ('XGB', xgb), ('LGBM',lgbm)],
                                 weights=[1, 1, 1, 1], voting ='hard').fit(x_train, y_train)
hard_voting_predicted_4 = voting_clf.predict(x_test)
hard_voting_predicted_4

Learning rate set to 0.073902
0:	learn: 1.0289654	total: 605us	remaining: 605ms
1:	learn: 0.9536345	total: 1.13ms	remaining: 563ms
2:	learn: 0.8977301	total: 1.69ms	remaining: 562ms
3:	learn: 0.8531481	total: 2.28ms	remaining: 567ms
4:	learn: 0.8065217	total: 2.74ms	remaining: 545ms
5:	learn: 0.7650036	total: 3.27ms	remaining: 542ms
6:	learn: 0.7272086	total: 3.76ms	remaining: 533ms
7:	learn: 0.6899499	total: 4.16ms	remaining: 516ms
8:	learn: 0.6575878	total: 4.76ms	remaining: 524ms
9:	learn: 0.6244861	total: 5.21ms	remaining: 516ms
10:	learn: 0.5922186	total: 5.75ms	remaining: 517ms
11:	learn: 0.5658850	total: 6.3ms	remaining: 519ms
12:	learn: 0.5389163	total: 6.8ms	remaining: 517ms
13:	learn: 0.5179312	total: 7.42ms	remaining: 522ms
14:	learn: 0.4959282	total: 8.1ms	remaining: 532ms
15:	learn: 0.4770140	total: 8.66ms	remaining: 533ms
16:	learn: 0.4575394	total: 9.22ms	remaining: 533ms
17:	learn: 0.4422733	total: 9.79ms	remaining: 534ms
18:	learn: 0.4256502	total: 10.3ms	remaining: 53

334:	learn: 0.0258170	total: 174ms	remaining: 346ms
335:	learn: 0.0257334	total: 175ms	remaining: 345ms
336:	learn: 0.0256511	total: 175ms	remaining: 345ms
337:	learn: 0.0255726	total: 176ms	remaining: 345ms
338:	learn: 0.0254941	total: 177ms	remaining: 344ms
339:	learn: 0.0254228	total: 177ms	remaining: 344ms
340:	learn: 0.0252809	total: 178ms	remaining: 344ms
341:	learn: 0.0251859	total: 178ms	remaining: 343ms
342:	learn: 0.0251034	total: 179ms	remaining: 342ms
343:	learn: 0.0250056	total: 179ms	remaining: 342ms
344:	learn: 0.0249046	total: 180ms	remaining: 342ms
345:	learn: 0.0248040	total: 180ms	remaining: 341ms
346:	learn: 0.0247127	total: 181ms	remaining: 340ms
347:	learn: 0.0246452	total: 181ms	remaining: 340ms
348:	learn: 0.0245766	total: 182ms	remaining: 339ms
349:	learn: 0.0245125	total: 182ms	remaining: 339ms
350:	learn: 0.0244349	total: 183ms	remaining: 338ms
351:	learn: 0.0243801	total: 183ms	remaining: 338ms
352:	learn: 0.0243017	total: 184ms	remaining: 337ms
353:	learn: 

689:	learn: 0.0109755	total: 354ms	remaining: 159ms
690:	learn: 0.0109582	total: 354ms	remaining: 158ms
691:	learn: 0.0109282	total: 355ms	remaining: 158ms
692:	learn: 0.0109152	total: 356ms	remaining: 158ms
693:	learn: 0.0108961	total: 356ms	remaining: 157ms
694:	learn: 0.0108810	total: 357ms	remaining: 157ms
695:	learn: 0.0108599	total: 357ms	remaining: 156ms
696:	learn: 0.0108373	total: 358ms	remaining: 156ms
697:	learn: 0.0108183	total: 358ms	remaining: 155ms
698:	learn: 0.0108019	total: 359ms	remaining: 155ms
699:	learn: 0.0107842	total: 360ms	remaining: 154ms
700:	learn: 0.0107690	total: 360ms	remaining: 154ms
701:	learn: 0.0107504	total: 361ms	remaining: 153ms
702:	learn: 0.0107350	total: 361ms	remaining: 153ms
703:	learn: 0.0107182	total: 362ms	remaining: 152ms
704:	learn: 0.0107041	total: 362ms	remaining: 152ms
705:	learn: 0.0106845	total: 363ms	remaining: 151ms
706:	learn: 0.0106672	total: 363ms	remaining: 151ms
707:	learn: 0.0106464	total: 364ms	remaining: 150ms
708:	learn: 

C:\ProgramData\Anaconda3\envs\tf_cpu\lib\site-packages\sklearn\ensemble\_voting.py:60: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.asarray([est.predict(X) for est in self.estimators_]).T


ValueError: could not broadcast input array from shape (175,1) into shape (175,)

In [26]:
voting_clf = VotingClassifier(estimators=[('random_foresting',ranfor), ('catboost', catboost), ('XGB', xgb), ('LGBM',lgbm)],
                                 weights=[1, 1, 1, 1], voting ='soft').fit(x_train, y_train)
soft_voting_predicted_4 = voting_clf.predict(x_test)
soft_voting_predicted_4

Learning rate set to 0.073902
0:	learn: 1.0289654	total: 604us	remaining: 604ms
1:	learn: 0.9536345	total: 1.14ms	remaining: 568ms
2:	learn: 0.8977301	total: 1.74ms	remaining: 577ms
3:	learn: 0.8531481	total: 2.3ms	remaining: 572ms
4:	learn: 0.8065217	total: 2.74ms	remaining: 545ms
5:	learn: 0.7650036	total: 3.36ms	remaining: 557ms
6:	learn: 0.7272086	total: 3.95ms	remaining: 561ms
7:	learn: 0.6899499	total: 4.34ms	remaining: 538ms
8:	learn: 0.6575878	total: 4.84ms	remaining: 533ms
9:	learn: 0.6244861	total: 5.3ms	remaining: 524ms
10:	learn: 0.5922186	total: 5.84ms	remaining: 525ms
11:	learn: 0.5658850	total: 6.45ms	remaining: 531ms
12:	learn: 0.5389163	total: 6.97ms	remaining: 529ms
13:	learn: 0.5179312	total: 7.54ms	remaining: 531ms
14:	learn: 0.4959282	total: 8.11ms	remaining: 533ms
15:	learn: 0.4770140	total: 8.65ms	remaining: 532ms
16:	learn: 0.4575394	total: 9.16ms	remaining: 530ms
17:	learn: 0.4422733	total: 9.73ms	remaining: 531ms
18:	learn: 0.4256502	total: 10.4ms	remaining: 5

341:	learn: 0.0251859	total: 177ms	remaining: 340ms
342:	learn: 0.0251034	total: 177ms	remaining: 340ms
343:	learn: 0.0250056	total: 178ms	remaining: 339ms
344:	learn: 0.0249046	total: 178ms	remaining: 339ms
345:	learn: 0.0248040	total: 179ms	remaining: 338ms
346:	learn: 0.0247127	total: 180ms	remaining: 338ms
347:	learn: 0.0246452	total: 180ms	remaining: 337ms
348:	learn: 0.0245766	total: 181ms	remaining: 337ms
349:	learn: 0.0245125	total: 181ms	remaining: 336ms
350:	learn: 0.0244349	total: 182ms	remaining: 336ms
351:	learn: 0.0243801	total: 182ms	remaining: 335ms
352:	learn: 0.0243017	total: 183ms	remaining: 335ms
353:	learn: 0.0241969	total: 183ms	remaining: 334ms
354:	learn: 0.0241307	total: 184ms	remaining: 334ms
355:	learn: 0.0240540	total: 184ms	remaining: 333ms
356:	learn: 0.0239319	total: 185ms	remaining: 333ms
357:	learn: 0.0238447	total: 185ms	remaining: 332ms
358:	learn: 0.0237665	total: 186ms	remaining: 332ms
359:	learn: 0.0236485	total: 186ms	remaining: 331ms
360:	learn: 

693:	learn: 0.0108961	total: 359ms	remaining: 158ms
694:	learn: 0.0108810	total: 359ms	remaining: 158ms
695:	learn: 0.0108599	total: 360ms	remaining: 157ms
696:	learn: 0.0108373	total: 361ms	remaining: 157ms
697:	learn: 0.0108183	total: 361ms	remaining: 156ms
698:	learn: 0.0108019	total: 362ms	remaining: 156ms
699:	learn: 0.0107842	total: 362ms	remaining: 155ms
700:	learn: 0.0107690	total: 363ms	remaining: 155ms
701:	learn: 0.0107504	total: 363ms	remaining: 154ms
702:	learn: 0.0107350	total: 364ms	remaining: 154ms
703:	learn: 0.0107182	total: 364ms	remaining: 153ms
704:	learn: 0.0107041	total: 365ms	remaining: 153ms
705:	learn: 0.0106845	total: 365ms	remaining: 152ms
706:	learn: 0.0106672	total: 366ms	remaining: 152ms
707:	learn: 0.0106464	total: 366ms	remaining: 151ms
708:	learn: 0.0106336	total: 367ms	remaining: 151ms
709:	learn: 0.0106030	total: 368ms	remaining: 150ms
710:	learn: 0.0105896	total: 368ms	remaining: 150ms
711:	learn: 0.0105737	total: 369ms	remaining: 149ms
712:	learn: 

array([0, 1, 2, 1, 0, 1, 2, 1, 0, 0, 2, 1, 1, 0, 1, 1, 0, 1, 1, 2, 1, 1,
       1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 2, 0, 1, 2, 1, 1, 2, 0, 1, 2, 1,
       1, 1, 1, 2, 1, 2, 0, 1, 0, 1, 1, 1, 2, 0, 1, 2, 0, 1, 2, 2, 2, 0,
       1, 0, 0, 1, 1, 1, 0, 0, 2, 1, 2, 1, 1, 1, 2, 1, 0, 1, 1, 1, 1, 1,
       2, 0, 1, 1, 2, 1, 1, 2, 0, 1, 0, 2, 0, 1, 1, 2, 0, 0, 2, 1, 0, 1,
       2, 1, 1, 1, 1, 0, 0, 2, 1, 2, 0, 1, 1, 2, 2, 1, 2, 1, 0, 1, 0, 0,
       1, 1, 1, 2, 0, 0, 1, 0, 0, 0, 2, 1, 1, 1, 0, 1, 2, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 2, 0, 1, 2, 2, 1, 0, 0, 2, 1, 1, 0, 1, 2, 2, 1, 1])

In [ ]:
submit['class'] = class_le.inverse_transform(hard_voting_predicted_4)
submit.to_csv('./answer/submit_hardvoting_4.csv', index=False)

In [28]:
submit['class'] = class_le.inverse_transform(soft_voting_predicted_4)
submit.to_csv('./answer/submit_softvoting_4.csv', index=False)